# Análise de Sessões de Trading com Decision Tree

## Objetivo do Projeto
Usar Machine Learning (Árvore de Decisão) para descobrir qual sessão de trading 
(Ásia, Londres ou Nova York) oferece as melhores oportunidades para EURUSD.

### O que vamos fazer:
1. **Preparação dos Dados**: Carregar e limpar dados históricos de EURUSD
2. **Feature Engineering**: Criar métricas de volatilidade por sessão
3. **Modelo de Classificação**: Prever qual sessão será a "vencedora" do dia
4. **Modelo de Regressão**: Prever o tamanho do movimento em pips

### Por que Decision Tree?
- Fácil de interpretar (você pode visualizar as regras)
- Não precisa de normalização dos dados
- Bom ponto de partida para aprender ML aplicado a trading

---

## 1. IMPORTAÇÃO DAS BIBLIOTECAS

Aqui importamos as ferramentas que vamos usar:
- **pandas**: Manipulação de dados em formato de tabela (DataFrame)
- **numpy**: Cálculos matemáticos e arrays
- **matplotlib**: Criação de gráficos
- **datetime**: Trabalhar com datas e horas

In [ ]:
# ============================================================
# IMPORTAÇÃO DAS BIBLIOTECAS
# ============================================================

import pandas as pd          # Manipulação de dados tabulares
import numpy as np           # Cálculos numéricos
import matplotlib.pyplot as plt  # Visualização de dados
import datetime              # Manipulação de datas
from datetime import date    # Trabalhar com datas específicas

%matplotlib inline           # Exibe gráficos direto no notebook

## 2. CARREGAMENTO DOS DADOS

Carregamos o arquivo CSV com dados históricos de EURUSD no timeframe H1 (1 hora).

**Parâmetros importantes:**
- `delimiter=';'`: O arquivo usa ponto-e-vírgula como separador (comum em arquivos europeus)

**Dica**: Sempre verifique o formato do seu arquivo CSV antes de carregar!

In [ ]:
# ============================================================
# CARREGAMENTO DOS DADOS
# ============================================================

# Lê o arquivo CSV com dados de EURUSD H1
# delimiter=';' porque o arquivo usa ponto-e-vírgula como separador
df = pd.read_csv("data/eurusd_h1.csv", delimiter=';')

In [ ]:
# Verifica as dimensões do DataFrame: (linhas, colunas)
# Esperamos ver algo como (22245, 5) - mais de 22 mil candles de 1 hora
df.shape

In [ ]:
# Visualiza as primeiras 5 linhas para entender a estrutura dos dados
# Colunas esperadas: Date, Open, High, Low, Close
df.head()

In [ ]:
# Estatísticas descritivas dos dados numéricos
# Mostra: count, mean, std, min, 25%, 50%, 75%, max
# Útil para identificar outliers ou erros nos dados
df.describe()

## 3. TRATAMENTO DE DATAS

A coluna 'Date' veio como texto (string). Precisamos converter para datetime
para poder extrair informações como hora, dia da semana, etc.

**ATENÇÃO ao formato da data!**
- `dayfirst=True`: Indica que a data está no formato DD/MM/YYYY (padrão brasileiro)
- Sem esse parâmetro, o pandas assume MM/DD/YYYY (padrão americano)

In [ ]:
# ============================================================
# CONVERSÃO DA COLUNA DATE PARA DATETIME
# ============================================================

# Converte a coluna Date de string para datetime
# dayfirst=True porque a data está no formato DD/MM/YYYY (brasileiro)
# IMPORTANTE: Sem esse parâmetro, 01/02/2015 seria interpretado como 2 de janeiro!
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [ ]:
# Confirma que a conversão funcionou
# Agora deve mostrar datetime64[ns] para a coluna Date
df.head()

In [ ]:
# Verifica os tipos de dados de cada coluna
# Date deve ser datetime64[ns], os demais float64
df.dtypes

## 4. VERIFICAÇÃO DO TIMEZONE DOS DADOS

**Por que isso é importante?**
- Precisamos saber em qual timezone os dados estão para classificar corretamente as sessões
- Se os dados estão em UTC, a sessão de Londres começa às 08:00
- Se estão em horário de Nova York, Londres começa às 03:00

**Método de verificação:**
1. Calculamos a volatilidade (Range = High - Low) para cada hora de um dia específico
2. O pico de volatilidade deve coincidir com a abertura de Londres
3. Se o pico está nas horas 8-9, os dados estão em UTC ✓

In [ ]:
# ============================================================
# VERIFICAÇÃO DO TIMEZONE
# ============================================================

# Extrai a hora de cada candle
hour = df["Date"].dt.hour
hour.head()

In [ ]:
# Cria uma data específica para análise
# Escolhemos um dia aleatório no meio da semana (quarta-feira)
date1 = date(2018, 7, 18)

In [ ]:
# Filtra apenas os candles desse dia específico
df_dia = df[df['Date'].dt.date == date1]
df_dia.head()

In [ ]:
# Adiciona coluna 'hour' ao DataFrame principal
# .dt.hour extrai apenas a hora do datetime
df['hour'] = df['Date'].dt.hour
df

In [ ]:
# Calcula o Range (volatilidade) para o dia de teste
# Range = High - Low (amplitude do candle)
df_dia['Range'] = df_dia['High'] - df_dia['Low']

In [ ]:
# Agrupa por hora e calcula a média do Range
# Isso mostra em qual hora a volatilidade é maior
df_dia.groupby(df_dia['Date'].dt.hour)['Range'].mean()

### Conclusão do Timezone

Se o pico de volatilidade está nas horas 8-9, **os dados estão em UTC**.

Isso faz sentido porque:
- Londres abre às 08:00 UTC
- A volatilidade aumenta logo após a abertura

---

## 5. CLASSIFICAÇÃO DAS SESSÕES

Agora que sabemos que os dados estão em UTC, podemos classificar cada candle
em uma das três sessões principais:

| Sessão | Horário UTC | Horas |
|--------|-------------|-------|
| **Ásia** | 00:00 - 07:59 | 0-7 |
| **Londres** | 08:00 - 15:59 | 8-15 |
| **Nova York** | 16:00 - 23:59 | 16-23 |

**Nota**: Essa é uma simplificação. Na realidade, há overlap entre Londres e NY (13:00-17:00 UTC).

In [ ]:
# ============================================================
# FUNÇÃO DE CLASSIFICAÇÃO DAS SESSÕES
# ============================================================

def session_classification(h):
    """
    Classifica a hora em uma das três sessões de trading.
    
    Parâmetros:
        h (int): Hora do dia (0-23)
    
    Retorna:
        str: Nome da sessão ('asia', 'london', ou 'ny')
    
    Sessões (em UTC):
        - Ásia: 00:00 - 07:59
        - Londres: 08:00 - 15:59  
        - Nova York: 16:00 - 23:59
    """
    if h >= 0 and h <= 7:
        return 'asia'
    elif h >= 8 and h <= 15:
        return 'london'
    else:
        return 'ny'

In [ ]:
# Aplica a função a cada linha do DataFrame
# .apply() executa a função para cada valor da coluna 'hour'
df['Session'] = df['hour'].apply(session_classification)
df.head(10)

## 6. AGREGAÇÃO DOS DADOS POR SESSÃO

Agora precisamos transformar os dados de H1 (24 candles por dia) em dados
por sessão (3 "candles" por dia: Ásia, Londres, NY).

**O que fazemos:**
1. Criamos uma coluna 'Data' apenas com a data (sem hora)
2. Agrupamos por Data + Sessão
3. Para cada grupo, calculamos:
   - **High**: máximo do período
   - **Low**: mínimo do período
   - **Open**: primeiro valor do período
   - **Close**: último valor do período

In [ ]:
# ============================================================
# CRIAÇÃO DA COLUNA 'DATA' (APENAS DATA, SEM HORA)
# ============================================================

# Extrai apenas a data (sem hora) para agrupar os candles do mesmo dia
# .dt.date retorna apenas a parte da data do datetime
df['Data'] = df['Date'].dt.date
df.head()

In [ ]:
# ============================================================
# AGREGAÇÃO DOS DADOS POR DIA E SESSÃO
# ============================================================

# Agrupa os dados por Data e Sessão, calculando OHLC de cada sessão
# - High: valor máximo (max) - maior preço da sessão
# - Low: valor mínimo (min) - menor preço da sessão  
# - Open: primeiro valor (first) - preço de abertura da sessão
# - Close: último valor (last) - preço de fechamento da sessão

df_session = df.groupby(['Data', 'Session']).agg({
    'High': 'max',      # Maior preço da sessão
    'Low': 'min',       # Menor preço da sessão
    'Open': 'first',    # Preço de abertura (primeiro candle)
    'Close': 'last'     # Preço de fechamento (último candle)
})

df_session.head(10)

In [ ]:
# Converte o índice (Data, Session) de volta para colunas
# Isso facilita a manipulação dos dados posteriormente
df_session.reset_index()

## 7. FEATURE ENGINEERING (Criação de Variáveis)

Aqui criamos as métricas que vão alimentar nosso modelo de ML:

| Feature | Fórmula | Significado |
|---------|---------|-------------|
| **Range** | High - Low | Volatilidade total da sessão (em pips) |
| **Movimento** | abs(Close - Open) | Movimento direcional (em pips) |
| **Ratio** | Movimento / Range | Eficiência do movimento (0 a 1) |

**Por que o Ratio é importante?**
- Ratio = 1: O preço foi direto de Open para Close (movimento limpo)
- Ratio = 0: O preço oscilou muito mas não saiu do lugar (movimento errático)

**Sessão "vencedora"**: A sessão com maior Movimento no dia.

In [ ]:
# ============================================================
# CRIAÇÃO DAS FEATURES (VARIÁVEIS PREDITORAS)
# ============================================================

# Range: Volatilidade total da sessão (High - Low)
# Representa quantos pips o preço variou na sessão
df_session['Range'] = df_session['High'] - df_session['Low']

# Movimento: Quanto o preço andou de forma direcional
# abs() garante valor positivo independente da direção (alta ou baixa)
df_session['Movimento'] = abs(df_session['Open'] - df_session['Close'])

# Ratio: Eficiência do movimento (quanto do range foi aproveitado)
# Ratio = 1: movimento limpo | Ratio próximo de 0: muito ruído
df_session['Ratio'] = df_session['Movimento'] / df_session['Range']

print(df_session)

## 8. IDENTIFICAÇÃO DA SESSÃO VENCEDORA

Para criar nosso target (variável que queremos prever), precisamos identificar
qual sessão teve o maior movimento em cada dia.

**Método:**
1. Para cada dia, encontramos o valor máximo de 'Movimento'
2. Marcamos como True a sessão que tem esse valor máximo
3. Criamos o target com o nome da sessão vencedora

In [ ]:
# ============================================================
# IDENTIFICAÇÃO DA SESSÃO VENCEDORA DE CADA DIA
# ============================================================

# transform('max') encontra o valor máximo de cada grupo (dia)
# e "espalha" esse valor para todas as linhas do grupo
# Isso permite comparar cada sessão com o máximo do dia
target = df_session.groupby('Data')['Movimento'].transform('max')
target2 = df_session.groupby('Data')['Ratio'].transform('max')

In [ ]:
# Cria colunas booleanas indicando se a sessão foi a vencedora
# Vencedora1: baseada no Movimento (nossa métrica principal)
# Vencedora2: baseada no Ratio (métrica alternativa)
df_session['Vencedora1'] = df_session['Movimento'] == target
df_session['Vencedora2'] = df_session['Ratio'] == target2

In [ ]:
# Visualiza o resultado - agora cada sessão tem True/False
# indicando se foi a vencedora do dia
df_session

In [ ]:
# Filtra dias onde a mesma sessão venceu em ambas as métricas
# Esses são os "dias mais claros" onde uma sessão dominou
df_session[(df_session['Vencedora1']) & (df_session['Vencedora2']) == True]

In [ ]:
# ============================================================
# CRIAÇÃO DO DATAFRAME DE VENCEDORAS
# ============================================================

# Filtra apenas as sessões que foram vencedoras (baseado em Movimento)
df_vencedoras = df_session[df_session['Vencedora1']]

# Reset do índice para facilitar manipulação
df_vencedoras = df_vencedoras.reset_index()

# Mantém apenas Data e Session (o que precisamos para o merge)
df_vencedoras = df_vencedoras[['Data', 'Session']]

In [ ]:
df_vencedoras.head()

In [ ]:
# ============================================================
# ANÁLISE DA DISTRIBUIÇÃO DAS SESSÕES VENCEDORAS
# ============================================================

# Conta quantas vezes cada sessão foi a vencedora
# Este é um insight MUITO importante para trading!
df_vencedoras['Session'].value_counts()

### 📊 Insight Importante!

A distribuição das sessões vencedoras mostra algo interessante:
- **Londres**: ~46% (mais frequente)
- **NY**: ~42%
- **Ásia**: ~12% (raramente vence)

**Conclusão para trading**: Para EURUSD, focar em Londres e NY faz muito mais sentido
do que operar na sessão asiática!

---

## 9. PREPARAÇÃO FINAL DOS DADOS

Agora precisamos:
1. Adicionar o dia da semana como feature
2. Fazer merge para adicionar o 'Target' (sessão vencedora) ao dataset principal

In [ ]:
# ============================================================
# ADIÇÃO DO DIA DA SEMANA
# ============================================================

# Reset do índice para ter Data e Session como colunas
df_session = df_session.reset_index()

# Extrai o dia da semana (0=Segunda, 1=Terça, ..., 6=Domingo)
# Isso pode revelar padrões como "NY performa melhor nas sextas"
df_session['DiaSemana'] = df_session['Data'].apply(lambda x: x.weekday())

In [ ]:
df_session.head()

In [ ]:
# ============================================================
# MERGE: ADICIONA O TARGET AO DATAFRAME PRINCIPAL
# ============================================================

# Merge junta duas tabelas baseado em uma coluna comum
# Aqui, para cada Data em df_session, pegamos a Session vencedora de df_vencedoras
# how='left' mantém todas as linhas de df_session mesmo se não houver match
df_session = df_session.merge(df_vencedoras, on='Data', how='left')

In [ ]:
df_session.head()

In [ ]:
# ============================================================
# RENOMEAÇÃO DAS COLUNAS
# ============================================================

# Após o merge, temos Session_x (sessão atual) e Session_y (sessão vencedora)
# Renomeamos para nomes mais claros
df_session = df_session.rename(columns={
    'Session_x': 'Session',  # Sessão da linha atual
    'Session_y': 'Target'    # Sessão vencedora do dia (o que queremos prever)
})

In [ ]:
df_session.head()

## 10. MODELO 1: CLASSIFICAÇÃO

### Objetivo
Prever qual sessão será a vencedora do dia.

### Tipo de Problema
**Classificação Multiclasse** - Temos 3 classes possíveis: asia, london, ny

### Features (X) vs Target (y)
- **X (features)**: Range, Movimento, Ratio, DiaSemana
- **y (target)**: Target (nome da sessão vencedora)

### Train/Test Split
Dividimos os dados em:
- **70% para treino**: O modelo aprende os padrões
- **30% para teste**: Avaliamos se o modelo generaliza

**IMPORTANTE**: `shuffle=False` porque são dados temporais!
Não podemos usar dados do futuro para prever o passado.

In [ ]:
# ============================================================
# PREPARAÇÃO DOS DADOS PARA O MODELO DE CLASSIFICAÇÃO
# ============================================================

# Seleciona as features (variáveis preditoras)
# Usamos Range, Movimento, Ratio e DiaSemana para prever a sessão vencedora
x = df_session[['Range', 'Movimento', 'Ratio', 'DiaSemana']]

# Seleciona o target (o que queremos prever)
y = df_session['Target']

In [ ]:
# ============================================================
# DIVISÃO TREINO/TESTE
# ============================================================

from sklearn.model_selection import train_test_split

# Divide os dados em treino (70%) e teste (30%)
# test_size=0.3: 30% dos dados para teste
# shuffle=False: NÃO embaralha os dados (CRÍTICO para séries temporais!)
#   - Se shuffle=True, usaríamos dados de 2018 para prever 2015 (data leakage)
# random_state=42: semente para reproducibilidade

x_train, x_test, y_train, y_test = train_test_split(
    x, y, 
    test_size=0.3,
    shuffle=False,  # IMPORTANTE: manter ordem temporal
    random_state=42
)

In [ ]:
# Verifica as dimensões após o split
print(f"Treino: {x_train.shape[0]} amostras")
print(f"Teste: {x_test.shape[0]} amostras")

In [ ]:
# ============================================================
# TREINAMENTO DO MODELO DE CLASSIFICAÇÃO
# ============================================================

from sklearn.tree import DecisionTreeClassifier

# Cria o modelo de árvore de decisão
# max_depth=4: limita a profundidade da árvore para evitar overfitting
#              e facilitar a interpretação
mod_arvore = DecisionTreeClassifier(max_depth=4, random_state=42)

# Treina o modelo com os dados de treino
# .fit() é onde o modelo "aprende" os padrões
mod_arvore.fit(x_train, y_train)

In [ ]:
# ============================================================
# PREVISÕES
# ============================================================

# Faz previsões nos dados de treino (para comparar com teste)
y_pred_train = mod_arvore.predict(x_train)

# Faz previsões nos dados de teste (avaliação real)
y_pred_test = mod_arvore.predict(x_test)

In [ ]:
# ============================================================
# AVALIAÇÃO DO MODELO - ACURÁCIA
# ============================================================

from sklearn.metrics import accuracy_score

# Acurácia: percentual de previsões corretas
# Comparamos com baseline aleatório (33% para 3 classes)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print(f"Acurácia no Treino: {acc_train:.2%}")
print(f"Acurácia no Teste: {acc_test:.2%}")
print(f"\nBaseline aleatório: 33.33%")
print(f"Melhoria sobre baseline: {(acc_test - 0.3333) / 0.3333:.1%}")

In [ ]:
# ============================================================
# AVALIAÇÃO DO MODELO - MATRIZ DE CONFUSÃO
# ============================================================

from sklearn.metrics import confusion_matrix, classification_report

# Matriz de confusão mostra onde o modelo acerta e erra
# Linhas: valores reais | Colunas: valores previstos
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_test))
print("\n")

# Relatório detalhado com precision, recall e f1-score
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_test))

### Interpretação da Matriz de Confusão

A matriz de confusão mostra:
- **Diagonal principal**: Acertos (previsão = real)
- **Fora da diagonal**: Erros

**Métricas importantes:**
- **Precision**: Dos que o modelo disse que eram X, quantos realmente eram X?
- **Recall**: Dos que realmente eram X, quantos o modelo identificou?
- **F1-Score**: Média harmônica entre precision e recall

---

In [ ]:
# ============================================================
# VISUALIZAÇÃO DA ÁRVORE DE DECISÃO
# ============================================================

from sklearn.tree import plot_tree

# Cria figura grande para melhor visualização
plt.figure(figsize=(20, 10))

# Plota a árvore de decisão
# feature_names: nomes das features para aparecer nos nós
# class_names: nomes das classes para aparecer nas folhas
# filled=True: colore os nós de acordo com a classe majoritária
# rounded=True: bordas arredondadas
plot_tree(
    mod_arvore,
    feature_names=x.columns.tolist(),
    class_names=['asia', 'london', 'ny'],
    filled=True,
    rounded=True,
    fontsize=10
)

plt.title("Árvore de Decisão - Classificação de Sessões EURUSD")
plt.tight_layout()
plt.show()

## 11. MODELO 2: REGRESSÃO

### Objetivo
Prever o tamanho do movimento (em pips) de cada sessão.

### Tipo de Problema
**Regressão** - O target é um valor contínuo (não categorias)

### Features (X2) vs Target (y2)
- **X2 (features)**: DiaSemana, Session (convertida para número)
- **y2 (target)**: Movimento (valor contínuo)

### CUIDADO: Evitando Data Leakage!
NÃO usamos Range ou Ratio como features porque:
- Range e Movimento são calculados ao mesmo tempo
- Usar Range para prever Movimento seria "trapacear"
- Na vida real, você não sabe o Range antes da sessão acabar!

In [ ]:
# ============================================================
# PREPARAÇÃO DOS DADOS PARA REGRESSÃO
# ============================================================

# Converte Session para número (o modelo precisa de dados numéricos)
# asia=0, london=1, ny=2
session_map = {'asia': 0, 'london': 1, 'ny': 2}
df_session['Session_num'] = df_session['Session'].map(session_map)

# Features: apenas DiaSemana e Session_num
# NÃO usamos Range/Ratio para evitar data leakage!
x2 = df_session[['DiaSemana', 'Session_num']]

# Target: Movimento (o que queremos prever)
y2 = df_session['Movimento']

In [ ]:
# ============================================================
# DIVISÃO TREINO/TESTE PARA REGRESSÃO
# ============================================================

x_train2, x_test2, y_train2, y_test2 = train_test_split(
    x2, y2,
    test_size=0.3,
    shuffle=False,  # Mantém ordem temporal
    random_state=42
)

In [ ]:
# ============================================================
# TREINAMENTO DO MODELO DE REGRESSÃO
# ============================================================

from sklearn.tree import DecisionTreeRegressor

# DecisionTreeRegressor para valores contínuos
# max_depth=4: mesma ideia de limitar complexidade
mod_arvore2 = DecisionTreeRegressor(max_depth=4, random_state=42)

# Treina o modelo
mod_arvore2.fit(x_train2, y_train2)

In [ ]:
# ============================================================
# PREVISÕES DO MODELO DE REGRESSÃO
# ============================================================

y_pred_train2 = mod_arvore2.predict(x_train2)
y_pred_test2 = mod_arvore2.predict(x_test2)

In [ ]:
# ============================================================
# VISUALIZAÇÃO DA ÁRVORE DE REGRESSÃO
# ============================================================

plt.figure(figsize=(20, 10))

plot_tree(
    mod_arvore2,
    feature_names=['DiaSemana', 'Session_num'],
    filled=True,
    rounded=True,
    fontsize=10
)

plt.title("Árvore de Decisão - Regressão do Movimento EURUSD")
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# COMPARAÇÃO: VALORES REAIS VS PREVISTOS
# ============================================================

# Cria DataFrame para comparar previsões com valores reais
df2_aval = pd.DataFrame({
    'Real': y_test2,
    'Previsto': y_pred_test2
})
df2_aval

In [ ]:
# ============================================================
# AVALIAÇÃO DO MODELO DE REGRESSÃO
# ============================================================

from sklearn import metrics

# MAE (Mean Absolute Error): Erro médio em valor absoluto
# Quanto menor, melhor
mae = metrics.mean_absolute_error(y_test2, y_pred_test2)
print(f'Mean Absolute Error: {mae}')

# RMSE (Root Mean Squared Error): Penaliza mais erros grandes
rmse = np.sqrt(metrics.mean_squared_error(y_test2, y_pred_test2))
print(f'Root Mean Squared Error: {rmse}')

In [ ]:
# ============================================================
# VERIFICAÇÃO DE OVERFITTING
# ============================================================

# Compara erro no treino vs teste
# Se erro_treino << erro_teste: OVERFITTING (modelo decorou os dados)
# Se erro_treino ≈ erro_teste: OK (modelo generaliza bem)

mae_train = metrics.mean_absolute_error(y_train2, mod_arvore2.predict(x_train2))
mae_test = metrics.mean_absolute_error(y_test2, y_pred_test2)

print(f'MAE - Treinamento: {mae_train}')
print(f'MAE - Teste: {mae_test}')

if mae_train * 1.1 < mae_test:
    print("\n⚠️ ATENÇÃO: Possível overfitting!")
else:
    print("\n✅ Modelo generaliza bem (sem overfitting)")

In [ ]:
# ============================================================
# INTERPRETAÇÃO DO ERRO
# ============================================================

# Calcula o MAE como percentual da média
# Isso dá uma ideia de quão "grande" é o erro em termos relativos

media_movimento = y_test2.mean()
print(f"Média do Movimento: {media_movimento}")
print()

erro_percentual = mae / media_movimento * 100
print(f'O percentual do MAE em relação à média da base:')
print(f'{erro_percentual:.2f}%')
print()
print(f"Interpretação: O modelo erra em média {erro_percentual:.1f}% do movimento real.")

## 12. CONCLUSÕES E PRÓXIMOS PASSOS

### Resultados do Modelo de Classificação
- **Acurácia**: ~41% (vs 33% aleatório)
- O modelo é ~24% melhor que chute aleatório
- Londres e NY são mais previsíveis que Ásia

### Resultados do Modelo de Regressão
- **MAE**: ~7.5% do movimento médio
- Sem overfitting (erro treino ≈ erro teste)

### Insights para Trading
1. **Foque em Londres e NY** para EURUSD - juntas vencem 88% dos dias
2. **Ásia é imprevisível** - apenas 12% de vitórias, modelo falha muito
3. **Padrões existem mas são fracos** - 41% não é suficiente para trading real

### Próximos Passos
1. Adicionar mais features (volatilidade do dia anterior, eventos econômicos)
2. Testar outros modelos (Random Forest, XGBoost)
3. Fazer backtesting com os sinais gerados
4. Incluir custos de transação na avaliação

---

**Leonardo Alves**: Dados, não opiniões! 